In [1]:
from rackuten_project_lib import *

In [122]:
# path : directory où se trouve les DF Final cleand containing Labels , classes number changed , image_name
path= 'D:/DataScienTest_WorkSpace/WS/Travail_Final/Datasets/'

# Callback outputs , hdf5 files 
path_output_models = 'D:/DataScienTest_WorkSpace/WS/Travail_Final/Trained_models/Conv1D/'

In [56]:
xtrain = pd.read_pickle(path + 'df_train_final.pkl')
xtrain.head(5)

,designation,description,productid,imageid,text,prdtypecode,labels,image_name
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,<NA>,3804725264,1263597046,olivia personalisiertes notizbuch seiten punkt...,0,adult books,image_1263597046_product_3804725264.jpg
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,<NA>,436067568,1008141237,journal arts art marche salon art asiatique pa...,18,magazines,image_1008141237_product_436067568.jpg
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,grand stylet ergonomique bleu gamepad nintendo...,2,video games accessories,image_938777978_product_201115110.jpg
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,<NA>,50418756,457047496,peluche donald europe disneyland marionnette d...,7,toys for children,image_457047496_product_50418756.jpg
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,guerre tuques luc grandeur veut organiser jeu ...,25,books,image_1077757786_product_278535884.jpg


In [57]:
ytrain = pd.DataFrame(xtrain['prdtypecode']).astype('int64')

In [58]:
unique, counts = np.unique(ytrain, return_counts=True)
dict(zip(unique, counts))

{0: 3116,
 1: 2508,
 2: 1681,
 3: 832,
 4: 2671,
 5: 3953,
 6: 764,
 7: 4870,
 8: 2070,
 9: 5045,
 10: 807,
 11: 2491,
 12: 3241,
 13: 5073,
 14: 4303,
 15: 803,
 16: 4993,
 17: 824,
 18: 4760,
 19: 4774,
 20: 1421,
 21: 4989,
 22: 2589,
 23: 10209,
 24: 2496,
 25: 2761,
 26: 872}

In [59]:
y = ytrain
#y

In [99]:
from sklearn.model_selection import train_test_split

In [100]:
text_train, text_test, y_train, y_test = train_test_split(xtrain['text'], y, test_size=0.2, random_state=1234)  

In [101]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = 20000)#default was 10000/best 20000

tokenizer.fit_on_texts(text_train)

X_train = tokenizer.texts_to_sequences(text_train)

X_test = tokenizer.texts_to_sequences(text_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

#print(text_train.shape)
#print(len(X_train))

In [102]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 400 #defautl was 250, best 400

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)

X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

#print("X_train", len(X_train))
#print("X_test", len(X_test))

In [103]:
embedding_dim = 100

In [104]:
from tensorflow.keras.layers import LSTMCell
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, GlobalMaxPooling1D, RNN, GRUCell, Dropout, concatenate, SpatialDropout1D, Conv1D

In [123]:
# For saving Callbacks and history results
#path_output_models = 'D:/DataScienTest_WorkSpace/WS/Rackupy_WS/Travail_Final/Trained_models/'
date_result = '13022022'
nbr_Epochs = 50
batch_size = 200

# CONV1D

In [127]:
model_conv1D = Sequential()

model_conv1D.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model_conv1D.add(layers.SpatialDropout1D(0.2))
model_conv1D.add(layers.Conv1D(64, 2, activation='relu'))
model_conv1D.add(layers.GlobalMaxPooling1D())
model_conv1D.add(layers.Dense(27, activation='softmax'))

         
model_conv1D.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 400, 100)          12494000  
                                                                 
 spatial_dropout1d_13 (Spati  (None, 400, 100)         0         
 alDropout1D)                                                    
                                                                 
 conv1d_13 (Conv1D)          (None, 399, 64)           12864     
                                                                 
 global_max_pooling1d_13 (Gl  (None, 64)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_13 (Dense)            (None, 27)                1755      
                                                                 
Total params: 12,508,619
Trainable params: 12,508,619

In [128]:
learning_rate =  0.001

model_conv1D.compile(optimizer= Adam(learning_rate= learning_rate), #  0.001 , default LR
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

***Callback***

In [129]:
%%time

file_name_checkpnt = 'checkpoint_Conv1D_' + date_result +  \
                                       '_LR_'+str(learning_rate) +'_' + str(nbr_Epochs)+ 'Epochs.h5'

#Sauvegarder les meilleurs poids du modèle au cours de l'entraînement :
checkpoint = ModelCheckpoint(filepath = path_output_models + file_name_checkpnt , 
                             monitor ='val_accuracy',
                             verbose = 1,
                             save_best_only = True, # (pour que le meilleur modèle ne soit pas écrasé)
                             save_weights_only = False,
                             mode ='max', #(permet de préciser si la métrique doit croître ou décroitre : ici on choisit 'min'
                                           #car la métrique est une perte à minimiser)
                             save_freq='epoch')
#Arrêter l'entraînement si le modèle n'évolue plus (très pratique pour ne pas gérer le
#nombre d'epoch) :
# A modifier pour avoir des 
early = EarlyStopping(monitor='val_loss',                    
                      min_delta = 0,
                      patience = 10, #(nombre d'epochs à attendre avant d'arrêter l'entraînement
                      restore_best_weights = True,
                      verbose = 1,
                      mode = 'min')

#lr_reduce = ReduceLROnPlateau(patience=2,
  #                             verbose=1)

callbacks = [checkpoint, early]

Wall time: 0 ns


In [130]:
path_output_models + file_name_checkpnt

'D:/DataScienTest_WorkSpace/WS/Travail_Final/Trained_models/Conv1D/checkpoint_Conv1D_13022022_LR_0.001_50Epochs.h5'

In [131]:
history_conv1D = model_conv1D.fit(X_train, y_train.values,
    batch_size = 200,
    epochs = nbr_Epochs,
    validation_data = [X_test, y_test.values],
    callbacks = [checkpoint, early])

Epoch 1/50
339/340 [============================>.] - ETA: 0s - loss: 1.9466 - accuracy: 0.4775
Epoch 00001: val_accuracy improved from -inf to 0.70425, saving model to D:/DataScienTest_WorkSpace/WS/Travail_Final/Trained_models/Conv1D\checkpoint_Conv1D_13022022_LR_0.001_50Epochs.h5
340/340 [==============================] - 5s 13ms/step - loss: 1.9446 - accuracy: 0.4781 - val_loss: 1.0733 - val_accuracy: 0.7043
Epoch 2/50
337/340 [============================>.] - ETA: 0s - loss: 0.8262 - accuracy: 0.7700
Epoch 00002: val_accuracy improved from 0.70425 to 0.77555, saving model to D:/DataScienTest_WorkSpace/WS/Travail_Final/Trained_models/Conv1D\checkpoint_Conv1D_13022022_LR_0.001_50Epochs.h5
340/340 [==============================] - 4s 12ms/step - loss: 0.8259 - accuracy: 0.7699 - val_loss: 0.7755 - val_accuracy: 0.7756
Epoch 3/50
339/340 [============================>.] - ETA: 0s - loss: 0.5595 - accuracy: 0.8395
Epoch 00003: val_accuracy improved from 0.77555 to 0.79133, saving mode

In [132]:
valid_score_conv1D = model_conv1D.evaluate(X_test, y_test.values, batch_size=batch_size)

85/85 [==============================] - 0s 3ms/step - loss: 0.6830 - accuracy: 0.8012


In [137]:
y_pred_conv1D = model_conv1D.predict(X_test)
#Appliquer la méthode argmax du tableau y_pred pour obtenir les classes les plus probable

y_pred_class_conv1D = y_pred_conv1D.argmax(axis = 1)

In [138]:
%%time
#Get the accuracy score
valid_score = model_conv1D.evaluate(X_test, y_test, batch_size=batch_size)

print("Accuracy: {:.2f}%".format(valid_score[1] * 100)) 
print("Loss: ",valid_score[0])
Conv1D_SC = np.round((valid_score[1] * 100), 2)

85/85 [==============================] - 0s 3ms/step - loss: 0.6830 - accuracy: 0.8012
Accuracy: 80.12%
Loss:  0.6829662919044495
Wall time: 278 ms


***Save  results***

In [152]:
file_name_hist = 'history_Conv1D_' + date_result +  \
                                       '_LR_'+str(learning_rate) + '_SC_' + str(Conv1D_SC) + '%_' + str(nbr_Epochs)+ 'Epochs'

In [157]:
#convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history_conv1D.history) 

# history to json:  
hist_json_file = path_output_models + file_name_hist + '.json'
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

# history to csv: 
hist_csv_file = path_output_models + file_name_hist +'.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

# history to pickle file
hist_df.to_pickle(path_output_models + file_name_hist +'.pkl')

***Save trained model***

In [160]:
file_name = 'history_Conv1D_' + date_result +  \
                                       '_LR_'+str(learning_rate) + '_SC_' + str(Conv1D_SC) + '%_' + str(nbr_Epochs)+ 'Epochs.hdf5'